In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

## Data Preparation

In [12]:
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import RandomizedSearchCV, train_test_split;
import numpy;
import pandas;
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier;
import sys;
from keras.utils import to_categorical;
REMOTE_MODULE_PATH = '/home/macbookretina/automatic-music-genre-classification/feature_extraction_deep_learning'
# LOCAL_MODULE_PATH = '/Users/macbookretina/Desktop/automatic-music-genre-classification/feature_extraction_deep_learning'
sys.path.insert(1, REMOTE_MODULE_PATH)
from custom_module.utilities import *

In [13]:
features = pandas.read_csv(MOUNTED_DATASET_PATH + '/data/cleaned_features.csv', index_col=0)
features

,genre_label,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,3,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,3,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [14]:
y = features.loc[:,'genre_label']
X = features.loc[:, features.columns.difference(['genre_label'])]
X

,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,mean_db4_cD3,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,-0.000069,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,0.000984,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,0.000019,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,-0.001993,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,0.000076,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,-0.000065,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,-0.000009,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,-0.000024,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,-0.000132,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [15]:
y.value_counts(normalize=True)

3    0.333434
1    0.333434
2    0.333131
Name: genre_label, dtype: float64

In [16]:
wavelet_predictors = X.filter(regex=(r'.+_db[458]{1}_.+'))
wavelet_predictor_labels = wavelet_predictors.columns.values
timbral_rhythmic_predictors = X.loc[:, features.columns.difference(
    numpy.append(wavelet_predictor_labels, 'genre_label'))]
timbral_rhythmic_predictors

,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_mfcc_1,mean_mfcc_10,mean_mfcc_11,mean_mfcc_12,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-80.892197,19.216515,-13.114045,11.498654,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-81.181618,20.319712,-2.532140,12.753151,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-202.881241,9.440219,-11.644846,7.070107,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-40.265057,16.794779,-3.511151,8.467217,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-140.261337,16.041454,-3.190496,11.844710,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,-285.722076,1.724312,3.334758,-1.919123,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,-325.716064,-5.354380,-14.410619,-4.959855,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,-264.064270,0.347791,5.412341,5.308122,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-178.481781,4.596525,6.592151,10.097037,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [17]:
predictors_all_label = timbral_rhythmic_predictors.columns.values
predictors_with_outliers_label = ['mean_beats', 'mean_spec_bw', 'mean_spec_centroid', 'mean_spec_rolloff',
                          'median_beats','median_spec_bw', 'median_spec_centroid', 
                          'median_spec_rolloff', 'std_beats', 'std_mfcc_1', 'var_spec_centroid',
                          'var_spec_rolloff'
                         ]
predictors_without_outliers_label = timbral_rhythmic_predictors.loc[
    :, timbral_rhythmic_predictors.columns.difference(predictors_with_outliers_label)].columns.values

In [18]:
preprocess_sp = standardization_pipeline(
    predictors_all_label,
    predictors_with_outliers_label,
    predictors_without_outliers_label)
preprocess_np = normalization_pipeline(
    predictors_all_label)

### Split & Transform Data

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    timbral_rhythmic_predictors, y, test_size=0.2, shuffle=True)

In [20]:
pipeline_estimator_sp = standardization_pipeline(
    predictors_all_label,
    predictors_with_outliers_label,
    predictors_without_outliers_label
).fit(X_train)
# X_train_sp = pipeline_estimator_sp.transform(X_train)
X_test_sp = pipeline_estimator_sp.transform(X_test)
X_test_sp.shape

(660, 109, 1)

In [21]:
pipeline_estimator_np = normalization_pipeline(
    predictors_all_label).fit(X_train)
# X_train_np = pipeline_estimator_np.transform(X_train)
X_test_np = pipeline_estimator_np.transform(X_test)
X_test_np.shape

(660, 109, 1)

In [22]:
# X_train_sp = standardization_pipeline(wavelet_predictor_labels, 
#                                    [], wavelet_predictor_labels
# #                                   ).fit_transform(X_train)
# X_test_sp = standardization_pipeline(
#     predictors_all_label,
#     predictors_with_outliers_label,
#     predictors_without_outliers_label
# ).fit_transform(X_test)
# X_test_sp.shape

In [23]:
# X_train_np = normalization_pipeline(wavelet_predictor_labels
# #                                   ).fit_transform(X_train)
# X_test_np = normalization_pipeline(predictors_all_label
#                                   ).fit_transform(X_test)
# X_test_np.shape

In [24]:
_, ncols = X_train.shape

In [25]:
# one hot encoding our labels
y_train = to_categorical(y_train, 4)
y_test = to_categorical(y_test, 4)

In [26]:
y_train.shape, y_test.shape

((2639, 4), (660, 4))

## Predictive Modelling

In [27]:
early_stopping_cb = callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)

In [28]:
param_dist = {
    'clf__n_hidden': [0,1,2,3,4,5],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100)
}

### FNN With Standardized Input

In [29]:
# wrap the function with keras wrapper
fnn_1 = KerasClassifier(build_fn=set_shape_create_model('fnn', ncols))

In [30]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', fnn_1)
])

In [31]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [32]:
# %%timeit
# fit model
search_1 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 109)               0         
_________________________________________________________________
dense (Dense)                (None, 99)                10890     
_________________________________________________________________
dense_1 (Dense)              (None, 99)                9900      
_________________________________________________________________
dense_2 (Dense)              (None, 99)                9900      
_________________________________________________________________
dense_3 (Dense)              (None, 99)                9900      
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 400       
Total params: 40,990
Trainable params: 40,990
Non-trainable params: 0
___________________________________________________________

Epoch 1/100
55/55 [==============================] - 0s 4ms/step - accuracy: 0.6407 - loss: 0.8824 - val_accuracy: 0.6682 - val_loss: 0.7741
Epoch 2/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.6947 - loss: 0.7184 - val_accuracy: 0.6697 - val_loss: 0.7672
Epoch 3/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.7146 - loss: 0.7017 - val_accuracy: 0.6697 - val_loss: 0.7299
Epoch 4/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.7339 - loss: 0.6697 - val_accuracy: 0.6606 - val_loss: 0.7769
Epoch 5/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.7192 - loss: 0.6590 - val_accuracy: 0.6803 - val_loss: 0.7282
Epoch 6/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.7379 - loss: 0.6292 - val_accuracy: 0.6788 - val_loss: 0.7218
Epoch 7/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.7487 - loss: 0.6156 - val_accuracy: 0.6864 - val_loss: 0.7301
Epoch 8/100
5

55/55 [==============================] - 0s 4ms/step - accuracy: 0.4707 - loss: 1.3410 - val_accuracy: 0.5515 - val_loss: 1.2843
Epoch 2/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.5645 - loss: 1.2229 - val_accuracy: 0.5621 - val_loss: 1.1443
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5742 - loss: 1.0955 - val_accuracy: 0.5636 - val_loss: 1.0286
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5787 - loss: 1.0038 - val_accuracy: 0.5803 - val_loss: 0.9553
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5895 - loss: 0.9439 - val_accuracy: 0.5894 - val_loss: 0.9093
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5969 - loss: 0.9018 - val_accuracy: 0.5894 - val_loss: 0.8763
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6089 - loss: 0.8694 - val_accuracy: 0.5970 - val_loss: 0.8518
Epoch 8/100
55/55 [======

Epoch 59/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7447 - loss: 0.6207 - val_accuracy: 0.6955 - val_loss: 0.6986
Epoch 60/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7476 - loss: 0.6204 - val_accuracy: 0.7030 - val_loss: 0.6937
Epoch 61/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7487 - loss: 0.6194 - val_accuracy: 0.6939 - val_loss: 0.6961
Epoch 62/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7504 - loss: 0.6170 - val_accuracy: 0.7000 - val_loss: 0.6917
Epoch 63/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7521 - loss: 0.6173 - val_accuracy: 0.6970 - val_loss: 0.6911
Epoch 64/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7504 - loss: 0.6158 - val_accuracy: 0.6939 - val_loss: 0.6947
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7533 - loss: 0.6155 - val_accuracy: 0.6955 - val_loss: 0.6910
Epoch 

Epoch 82/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7578 - loss: 0.6026 - val_accuracy: 0.6970 - val_loss: 0.7033
Epoch 83/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7601 - loss: 0.6025 - val_accuracy: 0.7045 - val_loss: 0.7017
Epoch 84/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7595 - loss: 0.6011 - val_accuracy: 0.6939 - val_loss: 0.7077
Epoch 85/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7612 - loss: 0.6018 - val_accuracy: 0.6924 - val_loss: 0.7115
Epoch 86/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7584 - loss: 0.6011 - val_accuracy: 0.6955 - val_loss: 0.7044
Epoch 87/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7612 - loss: 0.5996 - val_accuracy: 0.6955 - val_loss: 0.6967
Epoch 88/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7652 - loss: 0.5990 - val_accuracy: 0.7000 - val_loss: 0.7011
Epoch 

...

Epoch 6/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7307 - loss: 0.6476 - val_accuracy: 0.6758 - val_loss: 0.7417
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7398 - loss: 0.6472 - val_accuracy: 0.6712 - val_loss: 0.7273
Epoch 8/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7358 - loss: 0.6254 - val_accuracy: 0.6742 - val_loss: 0.7236
Epoch 9/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7369 - loss: 0.6244 - val_accuracy: 0.6773 - val_loss: 0.7075
Epoch 10/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7369 - loss: 0.6122 - val_accuracy: 0.6788 - val_loss: 0.7073
Epoch 11/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7489 - loss: 0.6057 - val_accuracy: 0.6879 - val_loss: 0.7114
Epoch 12/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7528 - loss: 0.5937 - val_accuracy: 0.7015 - val_loss: 0.7058
Epoch 13/1

Epoch 89/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7459 - loss: 0.6258 - val_accuracy: 0.6818 - val_loss: 0.7088
Epoch 90/100
55/55 [==============================] - 0s 1ms/step - accuracy: 0.7453 - loss: 0.6221
Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_25 (Flatten)         (None, 109)               0         
_________________________________________________________________
dense_91 (Dense)             (None, 4)                 440       
Total params: 440
Trainable params: 440
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [==============================] - 0s 4ms/step - accuracy: 0.3917 - loss: 1.4580 - val_accuracy: 0.5288 - val_loss: 1.1870
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5543 - loss: 1.1039 - val_accuracy: 0.5970 - val_loss: 1.064

55/55 [==============================] - 0s 2ms/step - accuracy: 0.6915 - loss: 0.7815 - val_accuracy: 0.6561 - val_loss: 0.8175
Epoch 11/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6943 - loss: 0.7693 - val_accuracy: 0.6606 - val_loss: 0.8152
Epoch 12/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6960 - loss: 0.7581 - val_accuracy: 0.6667 - val_loss: 0.8037
Epoch 13/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6960 - loss: 0.7512 - val_accuracy: 0.6667 - val_loss: 0.8023
Epoch 14/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7000 - loss: 0.7447 - val_accuracy: 0.6636 - val_loss: 0.8019
Epoch 15/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7057 - loss: 0.7361 - val_accuracy: 0.6545 - val_loss: 0.7833
Epoch 16/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7045 - loss: 0.7313 - val_accuracy: 0.6515 - val_loss: 0.7806
Epoch 17/100
55/55 

Epoch 68/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7295 - loss: 0.6457 - val_accuracy: 0.6742 - val_loss: 0.7175
Epoch 69/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7301 - loss: 0.6461 - val_accuracy: 0.6712 - val_loss: 0.7202
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7347 - loss: 0.6459 - val_accuracy: 0.6727 - val_loss: 0.7206
Epoch 71/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7307 - loss: 0.6449 - val_accuracy: 0.6712 - val_loss: 0.7220
Epoch 72/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7330 - loss: 0.6428 - val_accuracy: 0.6758 - val_loss: 0.7192
Epoch 73/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7347 - loss: 0.6438 - val_accuracy: 0.6712 - val_loss: 0.7208
Epoch 74/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7358 - loss: 0.6424 - val_accuracy: 0.6667 - val_loss: 0.7186
Epoch 

55/55 [==============================] - 0s 2ms/step - accuracy: 0.6659 - loss: 0.7932 - val_accuracy: 0.6727 - val_loss: 0.7572
Epoch 4/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.6869 - loss: 0.7380 - val_accuracy: 0.6848 - val_loss: 0.7305
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7068 - loss: 0.6968 - val_accuracy: 0.7015 - val_loss: 0.7020
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7216 - loss: 0.6662 - val_accuracy: 0.6939 - val_loss: 0.7213
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7324 - loss: 0.6461 - val_accuracy: 0.7030 - val_loss: 0.6869
Epoch 8/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7312 - loss: 0.6275 - val_accuracy: 0.6924 - val_loss: 0.7042
Epoch 9/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7381 - loss: 0.6243 - val_accuracy: 0.7015 - val_loss: 0.6870
Epoch 10/100
55/55 [=====

In [33]:
search_1.best_score_

0.7158014261618669

In [34]:
search_1.best_params_

{'clf__activation': 'selu',
 'clf__kernel_initializer': 'normal',
 'clf__n_hidden': 3,
 'clf__n_neurons': 94,
 'clf__optimizer': 'adam'}

In [35]:
search_1.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_with_outliers', Pipeline(memory=None,
     st..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f21846320b8>)])

### FNN With Normalized Input

In [36]:
# wrap the function with keras wrapper
fnn_2 = KerasClassifier(set_shape_create_model('fnn_np', ncols))

In [37]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', fnn_2)
])

In [38]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [39]:
# %%timeit
# fit model
search_2 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_31 (Flatten)         (None, 109)               0         
_________________________________________________________________
dense_112 (Dense)            (None, 4)                 440       
Total params: 440
Trainable params: 440
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [==============================] - 1s 10ms/step - accuracy: 0.3991 - loss: 1.3064 - val_accuracy: 0.3742 - val_loss: 1.1364
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.4588 - loss: 1.0864 - val_accuracy: 0.5470 - val_loss: 1.0554
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5713 - loss: 1.0218 - val_accuracy: 0.6015 - val_loss: 1.0024
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6020 - loss: 0.9

Epoch 1/100
55/55 [==============================] - 0s 4ms/step - accuracy: 0.3928 - loss: 1.3180 - val_accuracy: 0.3894 - val_loss: 1.1326
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.4753 - loss: 1.0878 - val_accuracy: 0.5439 - val_loss: 1.0546
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5702 - loss: 1.0231 - val_accuracy: 0.5667 - val_loss: 1.0045
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5981 - loss: 0.9801 - val_accuracy: 0.5939 - val_loss: 0.9677
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6197 - loss: 0.9452 - val_accuracy: 0.6197 - val_loss: 0.9390
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6259 - loss: 0.9203 - val_accuracy: 0.6061 - val_loss: 0.9231
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6345 - loss: 0.9011 - val_accuracy: 0.6348 - val_loss: 0.9005
Epoch 8/100
5

Epoch 59/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6993 - loss: 0.7324 - val_accuracy: 0.6682 - val_loss: 0.7574
Epoch 60/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7027 - loss: 0.7306 - val_accuracy: 0.6788 - val_loss: 0.7549
Epoch 61/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6993 - loss: 0.7313 - val_accuracy: 0.6742 - val_loss: 0.7554
Epoch 62/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7004 - loss: 0.7295 - val_accuracy: 0.6758 - val_loss: 0.7543
Epoch 63/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7010 - loss: 0.7284 - val_accuracy: 0.6742 - val_loss: 0.7548
Epoch 64/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.7038 - loss: 0.7265 - val_accuracy: 0.6561 - val_loss: 0.7610
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6998 - loss: 0.7284 - val_accuracy: 0.6727 - val_loss: 0.7524
Epoch 

Epoch 69/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6875 - loss: 0.7404 - val_accuracy: 0.6727 - val_loss: 0.7595
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6955 - loss: 0.7405 - val_accuracy: 0.6803 - val_loss: 0.7572
Epoch 71/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6943 - loss: 0.7394 - val_accuracy: 0.6879 - val_loss: 0.7593
Epoch 72/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6972 - loss: 0.7387 - val_accuracy: 0.6773 - val_loss: 0.7568
Epoch 73/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6932 - loss: 0.7385 - val_accuracy: 0.6788 - val_loss: 0.7554
Epoch 74/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6932 - loss: 0.7374 - val_accuracy: 0.6788 - val_loss: 0.7564
Epoch 75/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6989 - loss: 0.7373 - val_accuracy: 0.6879 - val_loss: 0.7578
Epoch 

Epoch 77/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6407 - loss: 0.8247 - val_accuracy: 0.6348 - val_loss: 0.8699
Epoch 78/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6424 - loss: 0.8238 - val_accuracy: 0.6364 - val_loss: 0.8690
Epoch 79/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6407 - loss: 0.8230 - val_accuracy: 0.6364 - val_loss: 0.8685
Epoch 80/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6430 - loss: 0.8223 - val_accuracy: 0.6364 - val_loss: 0.8675
Epoch 81/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6418 - loss: 0.8215 - val_accuracy: 0.6394 - val_loss: 0.8674
Epoch 82/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6424 - loss: 0.8207 - val_accuracy: 0.6394 - val_loss: 0.8667
Epoch 83/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.6430 - loss: 0.8199 - val_accuracy: 0.6424 - val_loss: 0.8664
Epoch 

...

55/55 [==============================] - 0s 2ms/step - accuracy: 0.3280 - loss: 1.0867 - val_accuracy: 0.3470 - val_loss: 1.0833
Epoch 50/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.3326 - loss: 1.0860 - val_accuracy: 0.3470 - val_loss: 1.0825
Epoch 51/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.3343 - loss: 1.0853 - val_accuracy: 0.3455 - val_loss: 1.0816
Epoch 52/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.3383 - loss: 1.0845 - val_accuracy: 0.3455 - val_loss: 1.0806
Epoch 53/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.3314 - loss: 1.0837 - val_accuracy: 0.3530 - val_loss: 1.0802
Epoch 54/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.3417 - loss: 1.0827 - val_accuracy: 0.3545 - val_loss: 1.0790
Epoch 55/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.3530 - loss: 1.0813 - val_accuracy: 0.3561 - val_loss: 1.0772
Epoch 56/100
55/55 

Epoch 1/100
55/55 [==============================] - 0s 5ms/step - accuracy: 0.3240 - loss: 1.3226 - val_accuracy: 0.3409 - val_loss: 1.2183
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.3314 - loss: 1.1662 - val_accuracy: 0.3409 - val_loss: 1.1273
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.3246 - loss: 1.1210 - val_accuracy: 0.3439 - val_loss: 1.1085
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.3258 - loss: 1.1090 - val_accuracy: 0.3439 - val_loss: 1.1022
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.3286 - loss: 1.1046 - val_accuracy: 0.3439 - val_loss: 1.0999
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.3314 - loss: 1.1021 - val_accuracy: 0.3439 - val_loss: 1.0984
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.3292 - loss: 1.1007 - val_accuracy: 0.3439 - val_loss: 1.0972
Epoch 8/100
5

Epoch 59/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.5577 - loss: 1.0002 - val_accuracy: 0.5697 - val_loss: 0.9914
Epoch 60/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5634 - loss: 0.9940 - val_accuracy: 0.5652 - val_loss: 0.9849
Epoch 61/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5685 - loss: 0.9879 - val_accuracy: 0.5742 - val_loss: 0.9785
Epoch 62/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5651 - loss: 0.9816 - val_accuracy: 0.5848 - val_loss: 0.9721
Epoch 63/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.5628 - loss: 0.9759 - val_accuracy: 0.5924 - val_loss: 0.9659
Epoch 64/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.5753 - loss: 0.9696 - val_accuracy: 0.5636 - val_loss: 0.9599
Epoch 65/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.5685 - loss: 0.9644 - val_accuracy: 0.5818 - val_loss: 0.9536
Epoch 

Epoch 65/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.4011 - loss: 1.0746 - val_accuracy: 0.4030 - val_loss: 1.0692
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.4017 - loss: 1.0731 - val_accuracy: 0.3970 - val_loss: 1.0675
Epoch 67/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.3955 - loss: 1.0716 - val_accuracy: 0.4121 - val_loss: 1.0660
Epoch 68/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.4261 - loss: 1.0698 - val_accuracy: 0.3970 - val_loss: 1.0639
Epoch 69/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.3898 - loss: 1.0682 - val_accuracy: 0.4545 - val_loss: 1.0626
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.4114 - loss: 1.0665 - val_accuracy: 0.4621 - val_loss: 1.0608
Epoch 71/100
55/55 [==============================] - 0s 2ms/step - accuracy: 0.4347 - loss: 1.0644 - val_accuracy: 0.4273 - val_loss: 1.0581
Epoch 

55/55 [==============================] - 0s 3ms/step - accuracy: 0.6648 - loss: 0.7891 - val_accuracy: 0.6636 - val_loss: 0.7827
Epoch 6/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.6722 - loss: 0.7642 - val_accuracy: 0.6394 - val_loss: 0.8296
Epoch 7/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.6733 - loss: 0.7580 - val_accuracy: 0.6909 - val_loss: 0.7444
Epoch 8/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.6864 - loss: 0.7439 - val_accuracy: 0.6500 - val_loss: 0.7782
Epoch 9/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.6881 - loss: 0.7338 - val_accuracy: 0.6515 - val_loss: 0.7578
Epoch 10/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.6824 - loss: 0.7273 - val_accuracy: 0.6091 - val_loss: 0.8053
Epoch 11/100
55/55 [==============================] - 0s 3ms/step - accuracy: 0.6926 - loss: 0.7219 - val_accuracy: 0.5985 - val_loss: 0.8319
Epoch 12/100
55/55 [===

In [40]:
search_2.best_score_

0.7025388249080712

In [41]:
search_2.best_params_

{'clf__activation': 'selu',
 'clf__kernel_initializer': 'glorot_uniform',
 'clf__n_hidden': 2,
 'clf__n_neurons': 76,
 'clf__optimizer': 'adam'}

In [27]:
search_2.best_score_

0.6854869241107364

In [28]:
search_2.best_params_

{'clf__activation': 'elu',
 'clf__kernel_initializer': 'glorot_uniform',
 'clf__n_hidden': 4,
 'clf__n_neurons': 67,
 'clf__optimizer': 'rmsprop'}

In [29]:
search_2.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_all', Pipeline(memory=None,
     steps=[('col..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fca180fa9b0>)])

### CNN With Standardized Input

In [42]:
param_dist = {
    'clf__n_hidden': [1, 2, 3],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100),
    'clf__filters': [16, 32, 64],
#     'clf__kernel_size': [3,4,5],
    'clf__dropout':[.1, .15, .2, .25]
}

In [43]:
# wrap the function with keras wrapper
cnn_1 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn', ncols))

In [44]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', cnn_1)
])

In [45]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [46]:
# %%timeit
# fit model
search_3 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 107, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 53, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51, 64)            6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 23, 96)            18528     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 11, 96)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 9, 128)            36992   

Epoch 1/100
55/55 [==============================] - 1s 14ms/step - accuracy: 0.4787 - loss: 1.0766 - val_accuracy: 0.5727 - val_loss: 0.9239
Epoch 2/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.5537 - loss: 0.9502 - val_accuracy: 0.5924 - val_loss: 0.8935
Epoch 3/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.5787 - loss: 0.9227 - val_accuracy: 0.6015 - val_loss: 0.8781
Epoch 4/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.5816 - loss: 0.9149 - val_accuracy: 0.6076 - val_loss: 0.8686
Epoch 5/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.6032 - loss: 0.8939 - val_accuracy: 0.6106 - val_loss: 0.8601
Epoch 6/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.6197 - loss: 0.8725 - val_accuracy: 0.6091 - val_loss: 0.8508
Epoch 7/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.6197 - loss: 0.8537 - val_accuracy: 0.6136 - val_loss: 0.8463
Epoch 

55/55 [==============================] - 1s 14ms/step - accuracy: 0.4938 - loss: 1.0732 - val_accuracy: 0.5833 - val_loss: 0.9203
Epoch 2/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.5665 - loss: 0.9519 - val_accuracy: 0.5985 - val_loss: 0.8909
Epoch 3/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.5909 - loss: 0.9283 - val_accuracy: 0.6136 - val_loss: 0.8825
Epoch 4/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.5972 - loss: 0.9043 - val_accuracy: 0.6091 - val_loss: 0.8619
Epoch 5/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6176 - loss: 0.8793 - val_accuracy: 0.6076 - val_loss: 0.8525
Epoch 6/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6119 - loss: 0.8821 - val_accuracy: 0.6167 - val_loss: 0.8443
Epoch 7/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6205 - loss: 0.8636 - val_accuracy: 0.6076 - val_loss: 0.8379
Epoch 8/100
55/55 

55/55 [==============================] - 1s 11ms/step - accuracy: 0.8192 - loss: 0.4588 - val_accuracy: 0.6788 - val_loss: 0.8399
Epoch 7/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.8573 - loss: 0.3746 - val_accuracy: 0.7091 - val_loss: 0.9317
Epoch 8/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.8846 - loss: 0.3052 - val_accuracy: 0.6758 - val_loss: 0.9538
Epoch 9/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.9215 - loss: 0.2084 - val_accuracy: 0.6758 - val_loss: 1.2776
Epoch 10/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.9323 - loss: 0.2055 - val_accuracy: 0.6515 - val_loss: 1.2666
Epoch 11/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.9505 - loss: 0.1489 - val_accuracy: 0.6697 - val_loss: 1.3591
Epoch 12/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.9460 - loss: 0.1589 - val_accuracy: 0.6333 - val_loss: 1.4329
Epoch 13/100
55

55/55 [==============================] - 0s 9ms/step - accuracy: 0.7732 - loss: 0.5608 - val_accuracy: 0.6955 - val_loss: 0.6941
Epoch 7/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8005 - loss: 0.5063 - val_accuracy: 0.7015 - val_loss: 0.6943
Epoch 8/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.8124 - loss: 0.4772 - val_accuracy: 0.7121 - val_loss: 0.7008
Epoch 9/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8351 - loss: 0.4499 - val_accuracy: 0.6924 - val_loss: 0.7107
Epoch 10/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8340 - loss: 0.4118 - val_accuracy: 0.6606 - val_loss: 0.8021
Epoch 11/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8675 - loss: 0.3610 - val_accuracy: 0.7045 - val_loss: 0.7238
Epoch 12/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8835 - loss: 0.3005 - val_accuracy: 0.7152 - val_loss: 0.7065
Epoch 13/100
55/55 [==

...

Epoch 88/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8101 - loss: 0.5146 - val_accuracy: 0.7106 - val_loss: 0.6867
Epoch 89/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8016 - loss: 0.5134 - val_accuracy: 0.7152 - val_loss: 0.6803
Epoch 90/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8158 - loss: 0.5114 - val_accuracy: 0.7121 - val_loss: 0.6816
Epoch 91/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8078 - loss: 0.5084 - val_accuracy: 0.7182 - val_loss: 0.6838
Epoch 92/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8027 - loss: 0.5055 - val_accuracy: 0.7167 - val_loss: 0.6857
Epoch 93/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8084 - loss: 0.5017 - val_accuracy: 0.7167 - val_loss: 0.6820
Epoch 94/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.8044 - loss: 0.5053 - val_accuracy: 0.7167 - val_loss: 0.6778
Epoch 

55/55 [==============================] - 0s 8ms/step - accuracy: 0.6767 - loss: 0.7423 - val_accuracy: 0.6727 - val_loss: 0.7736
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6943 - loss: 0.7277 - val_accuracy: 0.6667 - val_loss: 0.7662
Epoch 8/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7000 - loss: 0.7230 - val_accuracy: 0.6848 - val_loss: 0.7579
Epoch 9/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7011 - loss: 0.7079 - val_accuracy: 0.6788 - val_loss: 0.7527
Epoch 10/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6983 - loss: 0.7039 - val_accuracy: 0.6788 - val_loss: 0.7487
Epoch 11/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6994 - loss: 0.7022 - val_accuracy: 0.6712 - val_loss: 0.7448
Epoch 12/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.7176 - loss: 0.6947 - val_accuracy: 0.6803 - val_loss: 0.7404
Epoch 13/100
55/55 [==

Epoch 64/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7852 - loss: 0.5511 - val_accuracy: 0.6924 - val_loss: 0.6859
Epoch 65/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7812 - loss: 0.5434 - val_accuracy: 0.7106 - val_loss: 0.6862
Epoch 66/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7909 - loss: 0.5442 - val_accuracy: 0.6985 - val_loss: 0.6838
Epoch 67/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7847 - loss: 0.5351 - val_accuracy: 0.7136 - val_loss: 0.6867
Epoch 68/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7983 - loss: 0.5363 - val_accuracy: 0.6970 - val_loss: 0.6821
Epoch 69/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7898 - loss: 0.5371 - val_accuracy: 0.6985 - val_loss: 0.6865
Epoch 70/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7892 - loss: 0.5428 - val_accuracy: 0.6985 - val_loss: 0.6901
Epoch 

Epoch 1/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.5210 - loss: 1.0381 - val_accuracy: 0.5848 - val_loss: 0.8971
Epoch 2/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6182 - loss: 0.8615 - val_accuracy: 0.6212 - val_loss: 0.8438
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6511 - loss: 0.7926 - val_accuracy: 0.6364 - val_loss: 0.8236
Epoch 4/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6960 - loss: 0.7078 - val_accuracy: 0.6515 - val_loss: 0.8392
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7250 - loss: 0.6536 - val_accuracy: 0.6500 - val_loss: 0.7754
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7688 - loss: 0.5764 - val_accuracy: 0.6515 - val_loss: 0.8078
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.7977 - loss: 0.5120 - val_accuracy: 0.6515 - val_loss: 0.8766
Epoch 8/100


Epoch 90/100
83/83 [==============================] - 1s 8ms/step - accuracy: 0.7958 - loss: 0.5237 - val_accuracy: 0.7136 - val_loss: 0.6597
Epoch 91/100
83/83 [==============================] - 1s 8ms/step - accuracy: 0.8007 - loss: 0.5154 - val_accuracy: 0.7182 - val_loss: 0.6652
Epoch 92/100
83/83 [==============================] - 1s 8ms/step - accuracy: 0.7980 - loss: 0.5142 - val_accuracy: 0.7182 - val_loss: 0.6700
Epoch 93/100
83/83 [==============================] - 1s 8ms/step - accuracy: 0.7939 - loss: 0.5132 - val_accuracy: 0.7167 - val_loss: 0.6622
Epoch 94/100
83/83 [==============================] - 1s 8ms/step - accuracy: 0.7973 - loss: 0.5162 - val_accuracy: 0.7136 - val_loss: 0.6604
Epoch 95/100
83/83 [==============================] - 1s 8ms/step - accuracy: 0.7977 - loss: 0.5118 - val_accuracy: 0.7182 - val_loss: 0.6632
Epoch 96/100
83/83 [==============================] - 1s 8ms/step - accuracy: 0.7954 - loss: 0.5110 - val_accuracy: 0.7136 - val_loss: 0.6627
Epoch 

In [47]:
search_3.best_score_

0.7131488964656123

In [23]:
search_3.best_score_

0.7086017375283279

In [24]:
search_3.best_params_

{'clf__activation': 'relu',
 'clf__dropout': 0.1,
 'clf__filters': 64,
 'clf__kernel_initializer': 'normal',
 'clf__n_hidden': 1,
 'clf__n_neurons': 54,
 'clf__optimizer': 'adagrad'}

In [25]:
search_3.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_with_outliers', Pipeline(memory=None,
     st..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8ec8562f98>)])

### CNN With Normalized Input

In [48]:
# wrap the function with keras wrapper
cnn_2 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn_np', ncols))

In [49]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', cnn_1)
])

In [50]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [51]:
# %%timeit
# fit model
search_4 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_89 (Conv1D)           (None, 107, 32)           128       
_________________________________________________________________
max_pooling1d_89 (MaxPooling (None, 53, 32)            0         
_________________________________________________________________
conv1d_90 (Conv1D)           (None, 51, 64)            6208      
_________________________________________________________________
max_pooling1d_90 (MaxPooling (None, 25, 64)            0         
_________________________________________________________________
conv1d_91 (Conv1D)           (None, 23, 96)            18528     
_________________________________________________________________
max_pooling1d_91 (MaxPooling (None, 11, 96)            0         
_________________________________________________________________
conv1d_92 (Conv1D)           (None, 9, 128)            36992   

Epoch 1/100
55/55 [==============================] - 1s 22ms/step - accuracy: 0.3383 - loss: 1.1908 - val_accuracy: 0.3727 - val_loss: 1.1141
Epoch 2/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.3513 - loss: 1.1305 - val_accuracy: 0.5015 - val_loss: 1.0955
Epoch 3/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.3854 - loss: 1.1129 - val_accuracy: 0.5758 - val_loss: 1.0775
Epoch 4/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.4076 - loss: 1.0969 - val_accuracy: 0.5773 - val_loss: 1.0509
Epoch 5/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.4326 - loss: 1.0651 - val_accuracy: 0.5636 - val_loss: 1.0086
Epoch 6/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.5003 - loss: 1.0166 - val_accuracy: 0.5576 - val_loss: 0.9610
Epoch 7/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.5179 - loss: 0.9740 - val_accuracy: 0.5500 - val_loss: 0.9303
Epoch 

55/55 [==============================] - 1s 14ms/step - accuracy: 0.3301 - loss: 1.1755 - val_accuracy: 0.3409 - val_loss: 1.1125
Epoch 2/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.3483 - loss: 1.1390 - val_accuracy: 0.4455 - val_loss: 1.1042
Epoch 3/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.3511 - loss: 1.1212 - val_accuracy: 0.4333 - val_loss: 1.0920
Epoch 4/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.3693 - loss: 1.1115 - val_accuracy: 0.5758 - val_loss: 1.0722
Epoch 5/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.4176 - loss: 1.0855 - val_accuracy: 0.5818 - val_loss: 1.0414
Epoch 6/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.4403 - loss: 1.0643 - val_accuracy: 0.5697 - val_loss: 1.0075
Epoch 7/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.4972 - loss: 1.0219 - val_accuracy: 0.5712 - val_loss: 0.9642
Epoch 8/100
55/55 

Epoch 1/100
55/55 [==============================] - 1s 15ms/step - accuracy: 0.4360 - loss: 1.0904 - val_accuracy: 0.6242 - val_loss: 0.8249
Epoch 2/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.6214 - loss: 0.8512 - val_accuracy: 0.6561 - val_loss: 0.7766
Epoch 3/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.6538 - loss: 0.7885 - val_accuracy: 0.6697 - val_loss: 0.7488
Epoch 4/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.6987 - loss: 0.7304 - val_accuracy: 0.6318 - val_loss: 0.8752
Epoch 5/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.7112 - loss: 0.6880 - val_accuracy: 0.6773 - val_loss: 0.7315
Epoch 6/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.7305 - loss: 0.6514 - val_accuracy: 0.6939 - val_loss: 0.7244
Epoch 7/100
55/55 [==============================] - 1s 12ms/step - accuracy: 0.7413 - loss: 0.5948 - val_accuracy: 0.7061 - val_loss: 0.7140
Epoch 

Epoch 1/100
55/55 [==============================] - 1s 14ms/step - accuracy: 0.3994 - loss: 1.1233 - val_accuracy: 0.6273 - val_loss: 0.8304
Epoch 2/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6051 - loss: 0.8478 - val_accuracy: 0.6727 - val_loss: 0.7732
Epoch 3/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6744 - loss: 0.7638 - val_accuracy: 0.6667 - val_loss: 0.7567
Epoch 4/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6989 - loss: 0.6919 - val_accuracy: 0.6288 - val_loss: 0.8313
Epoch 5/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.7210 - loss: 0.6711 - val_accuracy: 0.6439 - val_loss: 0.8215
Epoch 6/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.7455 - loss: 0.6224 - val_accuracy: 0.6848 - val_loss: 0.8297
Epoch 7/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.7551 - loss: 0.5893 - val_accuracy: 0.6909 - val_loss: 0.7474
Epoch 

...

Epoch 63/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6725 - loss: 0.7564 - val_accuracy: 0.6576 - val_loss: 0.7885
Epoch 64/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6799 - loss: 0.7511 - val_accuracy: 0.6636 - val_loss: 0.7766
Epoch 65/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6697 - loss: 0.7544 - val_accuracy: 0.6545 - val_loss: 0.7907
Epoch 66/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6720 - loss: 0.7537 - val_accuracy: 0.6561 - val_loss: 0.7991
Epoch 67/100
55/55 [==============================] - 1s 9ms/step - accuracy: 0.6788 - loss: 0.7539 - val_accuracy: 0.6576 - val_loss: 0.7831
Epoch 68/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6850 - loss: 0.7464 - val_accuracy: 0.6576 - val_loss: 0.7815
Epoch 69/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6856 - loss: 0.7479 - val_accuracy: 0.6500 - val_loss: 0.7958
Epoch 

55/55 [==============================] - 0s 8ms/step - accuracy: 0.5063 - loss: 1.0150 - val_accuracy: 0.5742 - val_loss: 0.9737
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.5580 - loss: 0.9681 - val_accuracy: 0.5394 - val_loss: 0.9642
Epoch 4/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.5841 - loss: 0.9329 - val_accuracy: 0.6318 - val_loss: 0.9037
Epoch 5/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6017 - loss: 0.9151 - val_accuracy: 0.6197 - val_loss: 0.8835
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6074 - loss: 0.9001 - val_accuracy: 0.6152 - val_loss: 0.8762
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6170 - loss: 0.8874 - val_accuracy: 0.6121 - val_loss: 0.8622
Epoch 8/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6227 - loss: 0.8744 - val_accuracy: 0.6288 - val_loss: 0.8561
Epoch 9/100
55/55 [======

Epoch 60/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6733 - loss: 0.7659 - val_accuracy: 0.6530 - val_loss: 0.7765
Epoch 61/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6812 - loss: 0.7555 - val_accuracy: 0.6667 - val_loss: 0.7733
Epoch 62/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6767 - loss: 0.7582 - val_accuracy: 0.6606 - val_loss: 0.7710
Epoch 63/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6761 - loss: 0.7610 - val_accuracy: 0.6636 - val_loss: 0.7731
Epoch 64/100
55/55 [==============================] - 0s 8ms/step - accuracy: 0.6773 - loss: 0.7589 - val_accuracy: 0.6621 - val_loss: 0.7754
Epoch 65/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6824 - loss: 0.7522 - val_accuracy: 0.6636 - val_loss: 0.7702
Epoch 66/100
55/55 [==============================] - 0s 9ms/step - accuracy: 0.6847 - loss: 0.7581 - val_accuracy: 0.6576 - val_loss: 0.7719
Epoch 

55/55 [==============================] - 1s 12ms/step - accuracy: 0.3911 - loss: 1.1395 - val_accuracy: 0.5455 - val_loss: 0.9558
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - accuracy: 0.5452 - loss: 0.9386 - val_accuracy: 0.6212 - val_loss: 0.8332
Epoch 3/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6271 - loss: 0.8368 - val_accuracy: 0.6409 - val_loss: 0.8131
Epoch 4/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6504 - loss: 0.7869 - val_accuracy: 0.5924 - val_loss: 0.8846
Epoch 5/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6697 - loss: 0.7588 - val_accuracy: 0.6530 - val_loss: 0.7559
Epoch 6/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6953 - loss: 0.7215 - val_accuracy: 0.6879 - val_loss: 0.7299
Epoch 7/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.7197 - loss: 0.6684 - val_accuracy: 0.6667 - val_loss: 0.8023
Epoch 8/100
55/55 

Epoch 1/100
55/55 [==============================] - 1s 14ms/step - accuracy: 0.3489 - loss: 1.1617 - val_accuracy: 0.5015 - val_loss: 1.0434
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - accuracy: 0.5239 - loss: 0.9486 - val_accuracy: 0.6076 - val_loss: 0.8307
Epoch 3/100
55/55 [==============================] - 1s 10ms/step - accuracy: 0.6142 - loss: 0.8415 - val_accuracy: 0.6515 - val_loss: 0.7856
Epoch 4/100
55/55 [==============================] - 1s 11ms/step - accuracy: 0.6483 - loss: 0.7929 - val_accuracy: 0.6576 - val_loss: 0.7759
Epoch 5/100
55/55 [==============================] - 1s 10ms/step - accuracy: 0.6869 - loss: 0.7350 - val_accuracy: 0.6515 - val_loss: 0.7594
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - accuracy: 0.7051 - loss: 0.6964 - val_accuracy: 0.6727 - val_loss: 0.7463
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - accuracy: 0.7199 - loss: 0.6654 - val_accuracy: 0.6591 - val_loss: 0.7596
Epoch 

In [52]:
search_4.best_score_

0.7074649389515595

In [53]:
search_4.best_params_

{'clf__activation': 'relu',
 'clf__dropout': 0.1,
 'clf__filters': 64,
 'clf__kernel_initializer': 'normal',
 'clf__n_hidden': 1,
 'clf__n_neurons': 77,
 'clf__optimizer': 'adam'}

In [54]:
search_4.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['lpc_1', 'lpc_2', ..., 'var_spec_rolloff', 'var_zcr'], dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_all', Pipeline(memory=None,
     steps=[('col..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f21a83ae358>)])